In [9]:
import cv2
import numpy as np

In [10]:
img = cv2.imread("source.PNG")
gaussianImg = cv2.GaussianBlur(img, (9, 9), sigmaX=2, sigmaY=2)
borderSize = 20

In [11]:
hsvImg = cv2.cvtColor(gaussianImg, cv2.COLOR_BGR2HSV)
threshold, bwImg = cv2.threshold(hsvImg[:, :, 2], 0, 255, cv2.THRESH_OTSU)
ellipseKernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
morphedImg = cv2.morphologyEx(bwImg, cv2.MORPH_CLOSE, ellipseKernel)
distTransformImg = cv2.distanceTransform(morphedImg, cv2.DIST_L2, cv2.DIST_MASK_PRECISE)
distBorder = cv2.copyMakeBorder(distTransformImg, borderSize, borderSize, borderSize, borderSize, cv2.BORDER_CONSTANT | cv2.BORDER_ISOLATED, 0)

In [12]:
gap = 10
ellipseKernelTwo = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2 * (borderSize - gap) + 1, 2 * (borderSize - gap) + 1))
ellipseKernelTwo = cv2.copyMakeBorder(ellipseKernelTwo, gap, gap, gap, gap, cv2.BORDER_CONSTANT | cv2.BORDER_ISOLATED, 0)

In [13]:
distTransformTemp = cv2.distanceTransform(ellipseKernelTwo, cv2.DIST_L2, cv2.DIST_MASK_PRECISE)
nxcor = cv2.matchTemplate(distBorder, distTransformTemp, cv2.TM_CCOEFF_NORMED)

mn, mx, _, _ = cv2.minMaxLoc(nxcor)
threshold, peaks = cv2.threshold(nxcor, mx * 0.3, 255, cv2.THRESH_BINARY)
peaksScaledAbs = cv2.convertScaleAbs(peaks)

In [14]:
contours, hierarchy = cv2.findContours(peaksScaledAbs, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

In [15]:
for index in range(len(contours)):
    
    contour = contours[index]
    (x, y), radius = cv2.minEnclosingCircle(contour)
    radius = int(radius)
    
    (x, y, w, h) = cv2.boundingRect(contour)
    
    if len(contour) >= 5:
        ellipse = cv2.fitEllipse(contour)
        (center, axis, angle) = ellipse
        coordXContour, coordYContour = int(center[0]), int(center[1])
        print("Found!")
    

Found!
Found!
Found!
Found!
Found!
Found!
Found!
Found!
Found!
Found!
Found!
Found!
Found!
Found!
Found!


In [16]:
cv2.imshow('Raw Image', bwImg)
cv2.imshow('Morphed Ellipse', morphedImg)
cv2.imshow('Distance Transform', distTransformImg)
cv2.imshow('Distance Bordering', distBorder)
cv2.imshow('NXCor', nxcor)
cv2.imshow('Peaks Scaled Abs w/ Threshold', peaksScaledAbs)
cv2.waitKey(0)
cv2.destroyAllWindows()